# Please, click on "show widgets" and scroll down

In [1]:
%pip install matplotlib -qqq
import matplotlib.pyplot as plt

/usr/lib/python3/dist-packages/secretstorage/dhcrypto.py:15: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/usr/lib/python3/dist-packages/secretstorage/util.py:19: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
from population import *

In [3]:
import ipywidgets as widgets
from ipywidgets import Layout
from io import BytesIO
import colorsys
import pickle # package use to save data
from IPython.display import display, Image
import nbinteract as nbi

In [4]:
def f_a(S, Xb, l, h):
    if(abs(Xb-l) <= 10**-10):
        return 1./(S[0] + S[2])
    elif(abs(Xb-h) <= 10**-10):
        return 1./(S[0])
    else:
        print("Error!")
    return

def f_b(S, Xb, l, h):
    if(abs(Xb-l) <= 10**-10):
        return Xb/(S[1])
    elif(abs(Xb-h) <= 10**-10):
        return Xb/(S[1] + S[2])
    else:
        print("Error!")

def f_sab(S, Xb, l, h):
    if(abs(Xb-l) <= 10**-10):
        return 1./(S[0] + S[2])
    elif(abs(Xb-h) <= 10**-10):
        return h/(S[1] + S[2])
    else:
        print("Error!")

def F(S, Xb, l, h, alpha, r, m, d, k, normalization):
    A, B, SAB = S
    S2 = np.zeros((3))
    N = np.sum(S)
    if(normalization):
        S2[0] = r*N*f_a(S, Xb, l, h)*(1-d*N/k)*A-m*A
        S2[1] = r*N*f_b(S, Xb, l, h)*(1-d*N/k)*B-m*B
        S2[2] = r*N*f_sab(S, Xb, l, h)*(1-d*N/k)*SAB-m*SAB
    else:
        S2[0] = r*f_a(S, Xb, l, h)*(1-d*N/k)*A-m*A
        S2[1] = r*f_b(S, Xb, l, h)*(1-d*N/k)*B-m*B
        S2[2] = r*f_sab(S, Xb, l, h)*(1-d*N/k)*SAB-m*SAB
    return S2

def iterations(T, S_init, XXb, l, h, alpha, r, m, d, k, normalization):
    dt = T[1] - T[0]
    SS = np.zeros((len(T), 3))
    SS[0] = S_init
    for t in range(len(T)-1):
        k1 = F(SS[t], XXb[t], l, h, alpha, r, m, d, k, normalization)
        k2 = F(SS[t]+dt/2*k1, XXb[t], l, h, alpha, r, m, d, k, normalization)
        k3 = F(SS[t]+dt/2*k2, XXb[t], l, h, alpha, r, m, d, k, normalization)
        k4 = F(SS[t]+dt*k3, XXb[t+1], l, h, alpha, r, m, d, k, normalization)
        SS[t+1] = SS[t] + dt/6 * (k1+2*k2+2*k3+k4)
        #SS[t+1] = SS[t] + dt*F(SS[t], XXb[t], l, h, alpha, r, m, d, k, normalization) #euler explicit
    return SS

Strat = ["ia (always a)", "ib (always b)", "sab (sensitive)"]
Color = ["red", "green", "blue"]

In [5]:
class Select_box(widgets.VBox):
    def __init__(self, dashboard):
        
        import numpy as np
        
        self.dashboard = dashboard


        #initial point

        self.selection_ia0 = widgets.FloatLogSlider(min = np.log10(0.1), max = np.log10(1000), description = "N_{ia}^0", value =  1./3)
        self.selection_ia0.observe(dashboard.on_ia0_selected, names = "value")
        
        self.selection_ib0 = widgets.FloatLogSlider(min = np.log10(0.1), max = np.log10(1000), description = "N_{ib}^0", value =  1./3)
        self.selection_ib0.observe(dashboard.on_ib0_selected, names = "value")
        
        self.selection_sab0 = widgets.FloatLogSlider(min = np.log10(0.1), max = np.log10(1000), description = "N_{sab}^0", value =  1./3)
        self.selection_sab0.observe(dashboard.on_sab0_selected, names = "value")
        
        self.selection_l = widgets.FloatLogSlider(min = np.log10(0.1), max = np.log10(100), description = "l ", value =  0.6)
        self.selection_l.observe(dashboard.on_l_selected, names = "value")

        self.selection_h = widgets.FloatLogSlider(min = np.log10(0.1), max = np.log10(100), description = "h ", value =  2)
        self.selection_h.observe(dashboard.on_h_selected, names = "value")
        
        self.selection_alpha = widgets.FloatSlider(min = 0, max = 1, description = "alpha", value = 0.6)
        self.selection_alpha.observe(dashboard.on_alpha_selected, names = "value")
        
        
        self.selection_r = widgets.FloatSlider(min = 0, max = 4, description = "r", value = 3)
        self.selection_r.observe(dashboard.on_r_selected, names = "value")
        
        self.selection_m = widgets.FloatSlider(min = 0, max = 1, description = "m", value = 0.8)
        self.selection_m.observe(dashboard.on_m_selected, names = "value")
        
        self.selection_d = widgets.FloatSlider(min = 0, max = 1, description = "d", value = 0.4)
        self.selection_d.observe(dashboard.on_d_selected, names = "value")
        
        self.selection_k = widgets.FloatLogSlider(min = np.log10(1), max = np.log10(100000), description = "K", value =  1000)
        self.selection_k.observe(dashboard.on_k_selected, names = "value")
   

        self.selection_tFinal = widgets.FloatLogSlider(min = np.log10(10), max = np.log10(10000), description = "final time", value =  10)
        self.selection_tFinal.observe(dashboard.on_tFinal_selected, names = "value")
        
        self.selection_nbreStep = widgets.IntSlider(min = 100, max = 100000, description = "number of step", value =  100)
        self.selection_nbreStep.observe(dashboard.on_nbreStep_selected, names = "value")
        
        self.selection_period = widgets.IntSlider(min = 1, max = 100, description = "period", value = 5)
        self.selection_period.observe(dashboard.on_period_selected, names = "value")
            
        #self.selection_normalization = widgets.Dropdown(options=[('One', 1), ('Two', 2)], value=True, description='Number:')
        self.selection_normalization = widgets.Checkbox(value=True, description='Normalization', disabled=False, indent=False)
        self.selection_normalization.observe(dashboard.on_normalization_selected, names = "value")
        
        children = [
        self.selection_ia0,
        self.selection_ib0,
        self.selection_sab0,
        self.selection_l,
        self.selection_h,
        self.selection_alpha,
        self.selection_r,
        self.selection_m,
        self.selection_d,
        self.selection_k,
        self.selection_tFinal,
        self.selection_nbreStep,
        self.selection_period,
        self.selection_normalization,
        ]
        super().__init__(children, layout=Layout(width="100%"))

In [6]:
class Image_box(widgets.Box):
    def __init__(self, dashboard):
        #%pip install matplotlib
        #import matplotlib.pyplot as plt
        self.image = widgets.Image()
        self.dashboard = dashboard
        
        self.ia0 = self.dashboard.ia0
        self.ib0 = self.dashboard.ib0
        self.sab0 = self.dashboard.sab0
        
        self.l = self.dashboard.l
        self.h = self.dashboard.h
        self.alpha = self.dashboard.alpha
  
        self.r = self.dashboard.r
        self.m = self.dashboard.m
        self.d = self.dashboard.d
        self.k = self.dashboard.k
        
        self.tFinal = self.dashboard.tFinal
        self.nbreStep = self.dashboard.nbreStep
        self.period = self.dashboard.period
        
        self.normalization = self.dashboard.normalization
        
        #self.t_final = self.dashboard.tFinal
        #self.period_variable_source = self.dashboard.period
        
        
        self.print_image()
        
        image_container = widgets.Box([self.image], layout=Layout(width="100%"))
        
        children = [
   #     image_container,
        self.image,   
        ]
        super().__init__(children, layout=Layout(width="100%"))
        
    def print_image(self):
        """
        try:
            import matplotlib.pyplot as plt
        except:
             import pip
             pip._internal.main(['install', matplotlib])
             import matplotlib.pyplot as plt
        """
        
        self.t_final = self.tFinal
        self.period_variable_source = self.period
        
        
        T = np.linspace(0, self.t_final, self.nbreStep)
        Random_series_simple = np.random.binomial(1, self.alpha, size = self.nbreStep//self.period_variable_source) # generate again only if alpha changes
        Random_series = np.zeros(self.nbreStep)
        #for i in range(self.period_variable_source):
        #    Random_series[i::self.period_variable_source] = Random_series_simple
        for i in range(len(Random_series_simple)):
            Random_series[i*self.period_variable_source:i*self.period_variable_source+self.period_variable_source:] = Random_series_simple[i]

            
        XXb = self.l + (self.h-self.l)*Random_series

        SS = iterations(T, [self.ia0, self.ib0, self.sab0], XXb, self.l, self.h, self.alpha, self.r, self.m, self.d, self.k, self.normalization)

        Theory = np.array([1./(1+self.h), self.l/(1+self.l), (self.h-self.l)/((1+self.l)*(1+self.h))])
        
        #print("r(1+alpha*h + (1-alpha)l) - m ", self.r*(1+self.alpha*self.h + (1-self.alpha)*self.l) - self.m)

        plt.subplots(2,1, figsize = (15, 15))

        plt.subplot(2,1, 1)
        plt.title("Population model ODEs based on a simple Verhulst model\n", fontsize = 30)
        plt.suptitle("r(1+alpha*h + (1-alpha)l) - m = "+str(self.r*(1+self.alpha*self.h + (1-self.alpha)*self.l) - self.m), fontsize = 20)
        mmax = 1.1*np.max(np.max(SS))
        plt.fill_between(T, -0.1 , mmax, where=XXb==self.h, alpha = 0.2, color = "purple", label = "Xb = high")
        plt.fill_between(T, -0.1 , mmax, where=XXb==self.l, alpha = 0.2, color = "orange", label = "Xb = low")

        for i, strat in enumerate(Strat):
            plt.plot(T, SS[:,i], label = strat, color = Color[i])
        #    plt.plot([T[0], T[-1]], [Theory[i], Theory[i]], color = Color[i])
        #plt.ylim(0,1)
        plt.legend(fontsize = 20)
        plt.xlabel("time", fontsize = 30)
        plt.ylabel("Populations", fontsize = 30)

        plt.subplot(2,1, 2)
        plt.title("Population model ODEs based on a simple Verhulst model", fontsize = 30)
        for i, strat in enumerate(Strat):
            plt.plot(T, SS[:,i]/np.sum(SS, axis = 1), label = strat, color = Color[i])
            plt.plot([T[0], T[-1]], [Theory[i], Theory[i]], "--", color = Color[i])
        plt.fill_between(T, -0.1 , 1.1, where=XXb==self.h, alpha = 0.2, color = "purple", label = "Xb = high")
        plt.fill_between(T, -0.1 , 1.1, where=XXb==self.l, alpha = 0.2, color = "orange", label = "Xb = low")
        plt.ylim(-0.1,1.1)
        plt.legend(fontsize = 20)
        plt.xlabel("time", fontsize = 30)
        plt.ylabel("Frequency", fontsize = 30)
        plt.yticks([0, 1, 1./(1+self.h), self.l/(1+self.l), (self.h-self.l)/((1+self.l)*(1+self.h))], ["0", "1", "1./(1+h)", "l/(1+l)", "(h-l)/((1+l)*(1+h))"])


        #plt.show()


        image_file = BytesIO()
        #fig.savefig(fname = image_file)
        plt.savefig(fname = image_file)
        image_file.seek(0)
        image_data = image_file.read()
        self.image.value = image_data
#            self.image.width = 1500
#           self.image.height = 2000
        plt.close()

#          file = open(nom, "rb")
#         image = file.read()
        #plt.imshow(image)
#        self.image.value = image
 #       self.image.format = 'png'
        
        
        return
        

    def change_ia0(self, change):
        self.ia0 = change
        self.print_image()
        return
    
    def change_ib0(self, change):
        self.ib0 = change
        self.print_image()
        return
    
    def change_sab0(self, change):
        self.obl0 = change
        self.print_image()
        return
    
    def change_l(self, change):
        self.l = change
        self.print_image()
        return
    
    
    def change_h(self, change):
        self.h = change
        self.print_image()
        return
    
    def change_alpha(self, change):
        self.alpha = change
        self.print_image()
        return
    
    def change_r(self, change):
        self.r = change
        self.print_image()
        return
    
    
    def change_m(self, change):
        self.m = change
        self.print_image()
        return
    
    def change_d(self, change):
        self.d = change
        self.print_image()
        return
    
    def change_k(self, change):
        self.k = change
        self.print_image()
        return
    
    def change_tFinal(self, change):
        self.tFinal = change
        self.print_image()
        return
    
    def change_nbreStep(self, change):
        self.nbreStep = change
        self.print_image()
        return
    
    def change_period(self, change):
        self.period = change
        self.print_image()
        return
    
    def change_normalization(self, change):
        self.normalization = change
        self.print_image()
        return

In [7]:
class Dashboard(widgets.VBox):
    def __init__(self):

        self.ia0 = 1./3
        self.ib0 = 1./3
        self.sab0 = 1./3
        
        self.l = 0.6
        self.h = 2
        self.alpha = 0.6
          
        self.r = 3
        self.m = 0.8
        self.d = 0.4
        self.k = 1000
        
        self.tFinal = 10
        self.nbreStep = 100
        self.period = 5
        
        self.normalization = True
    
        self.select_box = Select_box(self)
    #    self.text_box = Text_box(self)
        self.image_box = Image_box(self)
        
        
        C1 = widgets.Box([self.image_box], layout=Layout(width="65%"))
        C2 = widgets.Box([self.select_box], layout=Layout(width="32%"))
        
        #rowA = widgets.Box([self.image_box, self.select_box], layout=Layout(width="100%"))
        rowA = widgets.HBox([C1, C2], layout=Layout(width="100%"))
        super().__init__([rowA], layout=Layout(width="100%"))
    

    
    
    def on_ia0_selected(self, change):
        self.ia0 = change["new"]
        self.image_box.change_ia0(change["new"])

    def on_ib0_selected(self, change):
        self.ib0 = change["new"]
        self.image_box.change_ib0(change["new"])

    def on_sab0_selected(self, change):
        self.sab0 = change["new"]
        self.image_box.change_sab0(change["new"])
        

    def on_l_selected(self, change):
        self.l = change["new"]
        self.image_box.change_l(change["new"])

    def on_h_selected(self, change):
        self.h = change["new"]
        self.image_box.change_h(change["new"])
        
    def on_alpha_selected(self, change):
        self.alpha = change["new"]
        self.image_box.change_alpha(change["new"])

        
    def on_r_selected(self, change):
        self.r = change["new"]
        self.image_box.change_r(change["new"])

    def on_m_selected(self, change):
        self.m = change["new"]
        self.image_box.change_m(change["new"])

    def on_d_selected(self, change):
        self.d = change["new"]
        self.image_box.change_d(change["new"])

    def on_k_selected(self, change):
        self.k = change["new"]
        self.image_box.change_k(change["new"])
        
    def on_tFinal_selected(self, change):
        self.tFinal = change["new"]
        self.image_box.change_tFinal(change["new"])
        
    def on_nbreStep_selected(self, change):
        self.nbreStep = change["new"]
        self.image_box.change_nbreStep(change["new"])
        
    def on_period_selected(self, change):
        self.period = change["new"]
        self.image_box.change_period(change["new"])
        
    def on_normalization_selected(self, change):
        self.normalization = change["new"]
        self.image_box.change_normalization(change["new"])

# Model
$\frac{d N_{ia}}{dt}   = R \cdot f_{ia}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) \cdot (1 - d\cdot N(t)/K) \cdot N_{ia}(t) - m \cdot N_{ia}(t)$

$\frac{d N_{ib}}{dt}   = R \cdot f_{ib}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) \cdot (1 - d\cdot N(t)/K) \cdot 
N_{ib}(t) - m \cdot N_{ib}(t)$

$\frac{d N_{sab}}{dt}   = R \cdot f_{sab}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) \cdot (1 - d\cdot N(t)/K) \cdot N_{sab}(t) - m \cdot N_{sab}(t)$


I chose $f_{ia}, f_{ib}$ and $f_{sab}$ as the quantity of food a player gets by choosing the corresponding strategy:

###### Normalized version

$f_{ia}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{N(t) \cdot 1}{N_{ia}(t) + N_{sab}(t)} & if & X_b = l \\
\frac{N(t) \cdot 1}{N_{ia}(t)} & if & X_b = h \\
\end{array}
\right.$ 

$f_{ib}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{N(t) \cdot l}{N_{ib}(t)} & if & X_b = l \\
\frac{N(t) \cdot h}{N_{ib}(t) +  N_{sab}(t)} & if & X_b = h \\
\end{array}
\right.$ 

$f_{sab}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{N(t) \cdot 1}{N_{ia}(t) + N_{sab}(t)} & if & X_b = l \\
\frac{N(t) \cdot h}{N_{ib}(t) + N_{sab}(t)} & if & X_b = h \\
\end{array}
\right.$ 

###### Non Normalized version

$f_{ia}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{1}{N_{ia}(t) + N_{sab}(t)} & if & X_b = l \\
\frac{1}{N_{ia}(t)} & if & X_b = h \\
\end{array}
\right.$ 

$f_{ib}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{l}{N_{ib}(t)} & if & X_b = l \\
\frac{h}{N_{ib}(t) +  N_{sab}(t)} & if & X_b = h \\
\end{array}
\right.$ 

$f_{sab}(N_{ia}(t), N_{ib}(t), N_{sab}(t), X_b(t)) = 
\left\{
\begin{array}{lll}
\frac{1}{N_{ia}(t) + N_{sab}(t)} & if & X_b = l \\
\frac{h}{N_{ib}(t) + N_{sab}(t)} & if & X_b = h \\
\end{array}
\right.$ 

On the following simulation, we can see that each frequency always converges to the theoretical prediction.

# Simulation

In [8]:
Dashboard()

Dashboard(children=(HBox(children=(Box(children=(Image_box(children=(Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x…

<ipython-input-6-520a5f0cfa2c>:92: RuntimeWarning: divide by zero encountered in true_divide
  plt.plot(T, SS[:,i]/np.sum(SS, axis = 1), label = strat, color = Color[i])
<ipython-input-6-520a5f0cfa2c>:92: RuntimeWarning: divide by zero encountered in true_divide
  plt.plot(T, SS[:,i]/np.sum(SS, axis = 1), label = strat, color = Color[i])
<ipython-input-6-520a5f0cfa2c>:92: RuntimeWarning: divide by zero encountered in true_divide
  plt.plot(T, SS[:,i]/np.sum(SS, axis = 1), label = strat, color = Color[i])


### If the simulation doesn't work (it may take a long time)
1. refresh
2. click again on show widget